In [1]:
# Import relevant libraries
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import html_text
import ast

In [2]:
# adding header to avoid trigerring captcha
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [3]:
# Define city for which the data is to be scrapped
city = 'Calgary,-AB_rb/'
# Define url 
url = 'https://www.zillow.com/homes/for_sale/'+city
# Define variable for number of website pages to be scrapped
num_of_pages=100
urls_dict={}
rs_dict={}

with requests.Session() as s:
    r = s.get(url, headers=req_headers)
    for i in range(1, num_of_pages):
        urls_dict['url{0}'.format(i)]='https://www.zillow.com/homes/for_sale/'+city+'/{0}_p/'.format(i)
        rs_dict['r{0}'.format(i)]=s.get(urls_dict['url{0}'.format(i)], headers=req_headers)    

In [4]:
#add contents of urls to soup variable from each url
soup = BeautifulSoup(r.content, 'html.parser')
soups_dict={}
for i in range(1, num_of_pages):
        soups_dict['soup{0}'.format(i)]= BeautifulSoup(rs_dict['r{0}'.format(i)].content, 'html.parser')

In [5]:
new_soup={'soup0':soup}
new_soup.update(soups_dict)


In [7]:
#create empty dataframes
dfs_dict={}
for i in range(num_of_pages):
        dfs_dict['df{0}'.format(i)]=pd.DataFrame()

# Read info from beatifulsoup parse
for key in new_soup:
    idx=list(new_soup).index(key)
    idx
    for i in new_soup[key]:
        soup = new_soup[key]
        address = soup.find_all (class_= 'list-card-addr')
        price = list(soup.find_all (class_='list-card-price'))
        beds = list(soup.find_all("ul", class_="list-card-details"))
        details = list(soup.find_all ('div', {'class': 'list-card-details'}))
        home_type = list(soup.find_all("ul", class_="list-card-details"))
        last_updated = soup.find_all ('div', {'class': 'list-card-top'})
        brokerage = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
        coord_tags = soup.findAll('script', {'type' : 'application/ld+json'})
        
        t = 0
        p = list(range(20))
        zipcode = []
        lat = []
        long = []
        home_type=[]
        for v in p:
            try:
                ex = coord_tags[t].text
                _dict = ast.literal_eval(ex)
                zipcode.append(_dict['address']['postalCode'])
                lat.append(_dict['geo']['latitude'])
                long.append(_dict['geo']['longitude'])
              
                t = t + 1
            except:
                t = t + 1
                continue
    
        #create dataframe columns out of variables
#         ex = coord_tags[0].text
#         _dict = ast.literal_eval(ex)
        
        df=dfs_dict['df{0}'.format(idx)]
        df['prices']= price
        df['address'] = address
        df['beds'] = beds 
        if len(beds)>len(url):
            zipcode.append(0)
            lat.append(0)
            long.append(0)
            home_type.append(0)

dfs=pd.DataFrame()
for key in dfs_dict:
    dfs=dfs.append(dfs_dict[key],ignore_index = True) 

In [8]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

C:\Users\saadi\AppData\Local\Temp/ipykernel_5192/3462665596.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [9]:
#convert columns to str
dfs['prices'] = dfs['prices'].astype('str')
dfs['address'] = dfs['address'].astype('str')
dfs['beds'] = dfs['beds'].astype('str')

In [10]:
#remove html tags
dfs['prices'] = dfs['prices'].replace('<div class="list-card-price">', ' ', regex=True)
dfs['prices'] = dfs['prices'].replace('</div>', ' ', regex=True)
dfs['prices'] = dfs['prices'].str.replace(r'\D', '')
dfs['address'] = dfs['address'].replace('<address class="list-card-addr">', ' ', regex=True)
dfs['address'] = dfs['address'].replace('</address>', ' ', regex=True)


C:\Users\saadi\AppData\Local\Temp/ipykernel_5192/3185812389.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  dfs['prices'] = dfs['prices'].str.replace(r'\D', '')


In [11]:
#remove html tags from beds column 
dfs['beds'] = dfs['beds'].replace('<ul class="list-card-details"><li class="">', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<ul class="list-card-details"><li>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->bds</', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->bds</abbr></li><li>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->ba</abbr></li><li>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->bd</abbr></li><li>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace(' abbr></li><li class="">', '|', regex=True)
dfs['beds'] = dfs['beds'].replace('Studio</li><li>', '0 ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->ba</', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace(' abbr></li><li class="">', '|', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace(' for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-  for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-  for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-  for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-  for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-  for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-  for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-  for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-  for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-  for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-  for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-  for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-  for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->bd</abbr></li><li class="">', '|', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- </li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->bd</abbr></li><li class="">', '|', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-  for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">-  for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- </li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- </li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- </li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->acres lot</abbr></li><li class="list-card-statusText">- Lot</li></ul>', '|', regex=True)
dfs['beds'] = dfs['beds'].replace('Studio</li><li class="">', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->acres lot</abbr></li><li class="list-card-statusText">- Lot</li></ul>', '|', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->acres lot</abbr></li><li class="list-card-statusText">- Lot</li></ul>', '|', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft lot</abbr></li><li class="list-card-statusText">- Lot</li></ul>', '|', regex=True)

In [12]:
dfs

,prices,address,beds
0,539900,"7412 W Fleetwood Dr SE, Calgary, AB T2H 0X1","3|2|1,293 House"
1,439999,"39 Saddlemont Mnr NE, Calgary, AB T3J 4Z5","2|2|1,460 House"
2,599900,"65 NW Auburn Glen Cmn SE, Calgary, AB T3M 0N2","3|3|1,205 House"
3,359900,"55 N Maranda Clos NE, Calgary, AB T2A 3E7",4|2|959 House
4,369900,"126 S Marquis Ln SE, Calgary, AB T3M 2G6","2|3|1,089 Townhouse"
5,650000,"99 W Arbour Stone Clos NW, Calgary, AB T3G 4T2","3|3|2,246 House"
6,375000,"233 N Marquis Ln SE, Calgary, AB T3M 2G6","2|3|1,109 Townhouse"
7,180000,"315 N Heritage Dr SE #406, Calgary, AB T2H 1N2",2|2|860 Apartment
8,559000,"493 W Auburn Crest Way SE, Calgary, AB T3M 1P9","5|4|1,738 House"
9,539900,"7412 W Fleetwood Dr SE, Calgary, AB T2H 0X1","3|2|1,293 House"


In [13]:
#split beds column into beds, bath and sq_feet
dfs[['beds','baths','sqft']] = dfs.beds.str.split(pat='|',expand=True)
#remove commas from sq_feet and convert to float
dfs.replace(',','', regex=True, inplace=True)
#convert column to float
dfs['prices'] = dfs['prices'].astype('float')

dfs

,prices,address,beds,baths,sqft
0,539900.0,7412 W Fleetwood Dr SE Calgary AB T2H 0X1,3,2,1293 House
1,439999.0,39 Saddlemont Mnr NE Calgary AB T3J 4Z5,2,2,1460 House
2,599900.0,65 NW Auburn Glen Cmn SE Calgary AB T3M 0N2,3,3,1205 House
3,359900.0,55 N Maranda Clos NE Calgary AB T2A 3E7,4,2,959 House
4,369900.0,126 S Marquis Ln SE Calgary AB T3M 2G6,2,3,1089 Townhouse
5,650000.0,99 W Arbour Stone Clos NW Calgary AB T3G 4T2,3,3,2246 House
6,375000.0,233 N Marquis Ln SE Calgary AB T3M 2G6,2,3,1109 Townhouse
7,180000.0,315 N Heritage Dr SE #406 Calgary AB T2H 1N2,2,2,860 Apartment
8,559000.0,493 W Auburn Crest Way SE Calgary AB T3M 1P9,5,4,1738 House
9,539900.0,7412 W Fleetwood Dr SE Calgary AB T2H 0X1,3,2,1293 House


Further cleaning of data

In [14]:
dfs['type']=""
dfs['type'] = dfs['sqft']
dfs

,prices,address,beds,baths,sqft,type
0,539900.0,7412 W Fleetwood Dr SE Calgary AB T2H 0X1,3,2,1293 House,1293 House
1,439999.0,39 Saddlemont Mnr NE Calgary AB T3J 4Z5,2,2,1460 House,1460 House
2,599900.0,65 NW Auburn Glen Cmn SE Calgary AB T3M 0N2,3,3,1205 House,1205 House
3,359900.0,55 N Maranda Clos NE Calgary AB T2A 3E7,4,2,959 House,959 House
4,369900.0,126 S Marquis Ln SE Calgary AB T3M 2G6,2,3,1089 Townhouse,1089 Townhouse
5,650000.0,99 W Arbour Stone Clos NW Calgary AB T3G 4T2,3,3,2246 House,2246 House
6,375000.0,233 N Marquis Ln SE Calgary AB T3M 2G6,2,3,1109 Townhouse,1109 Townhouse
7,180000.0,315 N Heritage Dr SE #406 Calgary AB T2H 1N2,2,2,860 Apartment,860 Apartment
8,559000.0,493 W Auburn Crest Way SE Calgary AB T3M 1P9,5,4,1738 House,1738 House
9,539900.0,7412 W Fleetwood Dr SE Calgary AB T2H 0X1,3,2,1293 House,1293 House


In [15]:
dfs["sqft"] = dfs["sqft"].str.split().str[0]
dfs

,prices,address,beds,baths,sqft,type
0,539900.0,7412 W Fleetwood Dr SE Calgary AB T2H 0X1,3,2,1293,1293 House
1,439999.0,39 Saddlemont Mnr NE Calgary AB T3J 4Z5,2,2,1460,1460 House
2,599900.0,65 NW Auburn Glen Cmn SE Calgary AB T3M 0N2,3,3,1205,1205 House
3,359900.0,55 N Maranda Clos NE Calgary AB T2A 3E7,4,2,959,959 House
4,369900.0,126 S Marquis Ln SE Calgary AB T3M 2G6,2,3,1089,1089 Townhouse
5,650000.0,99 W Arbour Stone Clos NW Calgary AB T3G 4T2,3,3,2246,2246 House
6,375000.0,233 N Marquis Ln SE Calgary AB T3M 2G6,2,3,1109,1109 Townhouse
7,180000.0,315 N Heritage Dr SE #406 Calgary AB T2H 1N2,2,2,860,860 Apartment
8,559000.0,493 W Auburn Crest Way SE Calgary AB T3M 1P9,5,4,1738,1738 House
9,539900.0,7412 W Fleetwood Dr SE Calgary AB T2H 0X1,3,2,1293,1293 House


In [16]:
dfs['type'] = dfs['type'].str.replace('\d+', '',regex=True)
dfs['type'] = dfs['type'].str.strip()
dfs

,prices,address,beds,baths,sqft,type
0,539900.0,7412 W Fleetwood Dr SE Calgary AB T2H 0X1,3,2,1293,House
1,439999.0,39 Saddlemont Mnr NE Calgary AB T3J 4Z5,2,2,1460,House
2,599900.0,65 NW Auburn Glen Cmn SE Calgary AB T3M 0N2,3,3,1205,House
3,359900.0,55 N Maranda Clos NE Calgary AB T2A 3E7,4,2,959,House
4,369900.0,126 S Marquis Ln SE Calgary AB T3M 2G6,2,3,1089,Townhouse
5,650000.0,99 W Arbour Stone Clos NW Calgary AB T3G 4T2,3,3,2246,House
6,375000.0,233 N Marquis Ln SE Calgary AB T3M 2G6,2,3,1109,Townhouse
7,180000.0,315 N Heritage Dr SE #406 Calgary AB T2H 1N2,2,2,860,Apartment
8,559000.0,493 W Auburn Crest Way SE Calgary AB T3M 1P9,5,4,1738,House
9,539900.0,7412 W Fleetwood Dr SE Calgary AB T2H 0X1,3,2,1293,House


Exporting to csv

In [17]:
dfs.to_csv('zillow_listing.csv', index=False)